In [2]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from PIL import Image

In [12]:

files = [
    'images/wiki_gradient.jpg',
    'images/wiki_white.jpg',
    'images/wiki_black.jpg',
    'images/wiki_transparency.png',
]


def save_bordered(fname):
    image = cv.imread(fname)
    
    new = np.zeros(shape=image.shape, dtype=np.uint8)
    black = np.zeros(shape=image.shape, dtype=np.uint8)
    
    #image = cv.cvtColor(image, cv.COLOR_RGB2RGBA)
    new_image = np.zeros(shape=image.shape, dtype=np.uint8)
    
    
    gray = cv.cvtColor(image, cv.COLOR_RGBA2GRAY)
    cnt_mask = np.zeros(shape=gray.shape, dtype=np.uint8)
    mask = np.zeros(shape=gray.shape, dtype=np.uint8)
    border = np.zeros(shape=gray.shape, dtype=np.uint8)
    
    
    gray = cv.bilateralFilter(gray, 7, 7, 7)
    
    canny = cv.Canny(gray, 20, 3)
    canny = cv.GaussianBlur(canny,(5,5),cv.BORDER_DEFAULT)
    
    contours, hierarchy = cv.findContours(canny, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cv.fillPoly(cnt_mask, pts=contours, color=1)
    cnt_mask = cv.GaussianBlur(cnt_mask,(5,5),cv.BORDER_DEFAULT)
    
    kernel = np.ones((3, 3), np.uint8)
    cnt_mask = cv.dilate(cnt_mask, kernel)
    kernel = np.ones((6, 6), np.uint8)
    cnt_mask = cv.erode(cnt_mask, kernel)
    
    outside_contours, hierarchy = cv.findContours(cnt_mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    outside_contours = sorted(outside_contours, key=cv.contourArea)
    largest_contour = outside_contours[-1]
    
    epsilon = 0.0002 * cv.arcLength(largest_contour, True)
    largest_contour = cv.approxPolyDP(largest_contour, epsilon, True)
    
    #cv.drawContours(image, [largest_contour], -1, (255,0,0), 2)
    cv.fillPoly(mask, pts=[largest_contour], color=255)
    
    border = cv.drawContours(border, [largest_contour], -1, (255,255,255), 50)
    border_filled = cv.fillPoly(border, pts=[largest_contour], color=255)
    
    
    #new_image = cv2.bitwise_or(new_image, solid_color, mask=border)
    
    solid_color = np.zeros(shape=new_image.shape, dtype=np.uint8)
    solid_color[:] = (0, 0, 255)
    #border_filled = cv.bitwise_and(solid_color, solid_color, mask=border_filled)
    
    new_image = cv.add(new_image, image, mask=mask)
    
    #mask = cv.bitwise_not(mask)
    
    solid_color = np.zeros(shape=new.shape, dtype=np.uint8)
    solid_color[:] = (0, 0, 255)
    
    new = cv.bitwise_or(new, solid_color,  mask=border_filled)
    new = cv.bitwise_and(new, new, mask=cv.bitwise_not(mask))
    
    blacked_image = cv.bitwise_or(black, image, mask=mask)
    
    #new = cv.add(new, blacked_image, mask=mask)
        
    bg = Image.fromarray(new)
    fg = Image.fromarray(new_image)
    mask = Image.fromarray(mask).convert("L")
    border_filled = Image.fromarray(border_filled).convert("L")
    
    #blank = fg.point(lambda _: 0)
    #fg = Image.composite(fg, blank, mask)
    
    bg.paste(fg, (0, 0), mask)
    bg.putalpha(border_filled)
    
    bg.save(fname.split('.')[0] + '_b.png')

for fname in files:
    save_bordered(fname)